# You look like notebook

## Setup

In [1]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf

## Inference

In [2]:
from yolov5.detect import run
from util import resize_face
from cnn.tf_cnn import predict

out = run('weights/yolov5/best.pt', source=0, nosave=True, conf_thres=0.6)
crop_im = resize_face(out['frame'], out['box'])


cv2.imshow('cropped face', crop_im)
cv2.waitKey(3000)
cv2.destroyWindow('cropped face')

img_array = tf.keras.preprocessing.image.img_to_array(crop_im)
img_batch = np.expand_dims(img_array, axis=0)
img_batch = img_batch/255

softmax_pred = predict(img_batch)

index = np.argmax(softmax_pred[0])

file_content = open("celebrities_idx.txt", "r").read()
celebrities_index = file_content.split(",")

print('You look like...', celebrities_index[index][2:-1])

c:\Users\cdetr\Documents\GitHub\you-look-like\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  2022-5-3 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)

Speed: 1.1ms pre-process, 16.4ms inference, 1.8ms NMS per image at shape (1, 3, 640, 640)


You look like... Hilary Clinton


## Train

In [ ]:
from cnn.tf_cnn import train

train(batch=32, epochs=20)

## Get data

In [ ]:
from get_data import create_directories, build_search_url, download_extended_page, get_all_items

# download celebrities pictures
f = open('people.txt')
keywords = f.read().splitlines()
print("Number of keywords:", len(keywords))

sufix = "face"

keywords = [keyword + " " + sufix for keyword in keywords]
search_keywords = [str(item) for item in keywords]

limit = 1400

main_directory = "data/cnn/download"

i_search_keywoard = 0

while i_search_keywoard < len(search_keywords):
	search_term = search_keywords[i_search_keywoard]

	subdirectory_name = search_term
	create_directories(main_directory, subdirectory_name)

	url = build_search_url(search_term)
	raw_html = download_extended_page(url)

	get_all_items(raw_html, main_directory, subdirectory_name, limit)

	i_search_keywoard += 1

In [ ]:
from util import is_face_centered

# crop faces
directory = 'data/cnn/download'
save_directory = 'data/cnn/download_cropped'
for celebrity in os.listdir(directory):
    if not os.path.exists(f'{save_directory}/{celebrity}'):
        os.mkdir(f'{save_directory}/{celebrity}')
    for img in os.listdir(f'{directory}/{celebrity}'):
        out = run('weights/yolov5/best.pt', source=f'{directory}/{celebrity}/{img}', nosave=True, conf_thres=0.8)
        if out['num_pred'] == 1:
            if is_face_centered(out['frame'], out['box']):
                crop_im = resize_face(out['frame'], out['box'])
                cv2.imwrite(f'{save_directory}/{celebrity}/{img}', crop_im)

In [ ]:
# augment and balance data
data_augmentor = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
]) 

directory = 'data/cnn/download_cropped'
for celebrity in os.listdir(directory):
    celebrity_images = os.listdir(f'{directory}/{celebrity}')
    num_of_img = len(celebrity_images)
    for i in range(500-num_of_img):
        idx = random.randint(0, num_of_img-1)
        img_path = f'{directory}/{celebrity}/{celebrity_images[idx]}'
        img = tf.keras.preprocessing.image.load_img(img_path)
        img = tf.cast(tf.expand_dims(img, 0), tf.float32)
        aug_img = data_augmentor(img)
        aug_img = np.reshape(aug_img.numpy(), (aug_img.shape[1], aug_img.shape[2], aug_img.shape[3]))
        cv2.imwrite(f'{img_path[:-4]}_augmented_{i}.jpg', cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB))